**1. INTRODUCTION**



In an increasingly data-driven environment, the ability to structure, store, and efficiently access information has become a key factor for the success of any information system. Databases not only enable the organization of large volumes of data but also facilitate decision-making, process automation, and continuous improvement across diverse fields such as industry, commerce, healthcare, and transportation.

This work presents a NoSQL database designed for the storage and analysis of information on aviation accidents that occurred in Argentina. Unlike traditional relational models, the NoSQL approach allows handling semi-structured and unstructured data, making it easier to integrate historical records, technical reports, geographic coordinates, operational metadata, and testimonies in varied formats.

The dataset used provides high informational value, as it enables an in-depth study of aviation accidents—a topic of great relevance for the safety and efficiency of air transportation. Analyzing these events is essential to identify patterns, establish preventive measures, and contribute to the continuous improvement of one of the most widely used means of transport worldwide.


**2. DATABASE ANALYSIS**

**2.1 Kaggle**

Kaggle is a web platform that brings together the largest Data Science community in the world, with more than 536,000 active members across 194 countries. It receives over 150,000 publications per month, providing all the essential tools and resources to maximize progress in Data Science. Kaggle, similar to DataScientest, offers a customizable Jupyter Notebook interface with no configuration required. It allows free access to GPUs and to a vast collection of datasets and code shared by the community.
On Kaggle, you will find the code and data you need to carry out your Data Science projects. There are more than 50,000 public datasets and 400,000 public notebooks available to everyone.

The dataset “Sucesos Aeronáutica Argentina” was obtained from Kaggle. In addition to downloading the .csv file, users can also explore graphical analyses and other insights related to the published datasets.

**2.2 MongoDB**

Once the .csv file was downloaded from the platform, the data was imported into MongoDB.

According to its official website, MongoDB is defined as:

“MongoDB is a document database that offers great scalability and flexibility, along with an advanced query and indexing model.”

From this definition, it can be concluded that MongoDB is a NoSQL database management system. This means it is a non-relational database that stores information in flexible documents instead of tables and rows, as in other systems.

**2.2.1 Data Import**

Once the data is imported, we observe a json object like this:





In [ ]:
{
  "_id": {
    "$oid": "690889ab526faf70466e6fef"
  },
  "Estado": "Cerrado",
  "Modelo": "AB-115 (Aero Boero)",
  "Matrícula": "LV-LPI",
  "Daños": "De importancia",
  "Fase": "Aterrizaje",
  "Lesiones": "Ninguno",
  "Fecha y hora de notificiacion": {
    "$date": "2023-06-10T05:51:00.000Z"
  },
  "fecha y hora de arribo": {
    "$date": "2023-06-10T04:48:00.000Z"
  },
  "Provincia": "BUENOS AIRES (BUE)",
  "Lugar": "Isla Martin Garcia",
  "tipo Lugar": "AD",
  "altura": null,
  "Oficina": null,
  "Canal": null,
  "Fabricante": "Aero Boero",
  "NroExp": "67901159/23",
  "Date": {
    "$date": "2023-06-10T00:00:00.000Z"
  },
  "ubicacion": {
    "type": "Point",
    "coordinates": [
      -58.249722222222225,
      -34.18222222222222
    ]
  },
  "fecha y hora de finalización": {
    "$date": "2025-11-09T20:41:33.515Z"
  }
}

As can be seen in the code above, the database “AeronauticalEvents” has been created and the Argentina collection has been added. It is modeled this way because it is planned to expand it in the future by adding new collections with datasets from other countries. Focusing on the Argentina collection, it is a fairly large collection of data, with a total of 2.1k documents.

Based on the previous code, the structure of a document or record is analyzed as follows:
- _id: Unique 12-byte identifier automatically assigned by MongoDB to the _id field of each document. This guarantees uniqueness even in distributed environments (ObjectId).
- Nro Exp: Case number (string).
- Date: Date of the event in DD/MM/YYYY format (string).
- Status: Status of the case, whether ongoing or closed (string).
- Model: Aircraft model (string).
- Registration: Aircraft identifier (string).
- Damage: Reported level of damage (string).
- Phase: Flight phase in which the event occurred (string).
- Lat-Lon: Accident coordinates (string).
- Injuries: Reported injuries (string).
- Notification Date/Time: Date and time of notification (string).
- Arrival Date/Time: Date and time of arrival at the accident site (string).
- Closure Date/Time: Date and time of case closure (string).
- Province: Province where the event occurred (string).
- Location: City or locality of the event (string).
- Location Type: Type of facility (e.g., Airport) (string).
- Altitude: Aircraft altitude at the time of the event (string).
- Office: Responsible office (string).
- Channel: Communication channel (string).
- Manufacturer: Aircraft manufacturer (string).

**2.4 Preprocessing and Validation**

Before performing queries and analyses on the dataset, it is necessary to review the quality and consistency of the data. The following are the main observations and actions taken:

**2.4.1 Altitude, Office, and Channel**
Several fields were identified with the value “S/D” (no data), which represents a non-standard way of encoding missing values. To facilitate the analysis, these values were transformed into null.

To achieve this, the console is accessed and, through a command, these values are corrected as shown below:


In [ ]:
db.Argentina.updateMany({"altura":"S/D"},{$set: {"altura":null}} ) 

Once the above code is executed in the console, the modification is successful,resulting in a total of 2069 updated documents.

This same change applies to the other fields that also contain the value of
“S/D”, which would be Office and Channel:

In [ ]:
db.Argentina.updateMany({"Oficina":"S/D"},{$set: {"Oficina":null}} ) 

In [ ]:
db.Argentina.updateMany({"Canal":"S/D"},{$set: {"Canal":null}} )

**2.4.2 'N° Exp'**

In all documents in the database, the 'N° Exp' field was treated as a subdocument with an empty key and not as a direct string.

This design causes problems when querying, filtering, or indexing 'N° Exp'. The goal is to make the file number a simple string, so that this field can be worked with more practically. To fix this problem, the following command is used:

In [ ]:
db.Argentina.find().forEach(doc => { 
if (doc["N° Exp"] && doc["N° Exp"][""] !== undefined) { 
db.Argentina.updateOne( 
{ _id: doc._id }, 
{ $set: { "N° Exp": doc["N° Exp"][""] } } 
); 
} 
}); 

Basically, this command aims to iterate through all the elements of the database. If there is a document with a file number and the empty subkey has a defined value, then it should replace the entire object with the found string value.

Another important point would be the field name, since the character “° ”,
makes filtering or querying a bit more complex for the analyst, so we proceed to change the field name:



In [ ]:
db.Argentina.updateMany({}, { $rename: { "N° Exp": "NroExp" } }) 

**2.4.3 Date**

As you can see, the document dates are a string, which is not ideal for processing date data. Therefore, this string is converted into a date using an aggregation pipeline:

In [ ]:
[ 
  { 
    $addFields: { 
      Date: { 
        $dateFromString: { 
          dateString: "$Fecha", 
          format: "%d/%m/%Y" 
        } 
      } 
    } 
  }, 
 
  {$merge: { 
    into: "Argentina", 
    whenMatched: 'merge', 
    whenNotMatched: 'discard' 
  }} 
] 

Now we proceed to remove the old "Date" field:

In [ ]:
db.Argentina.updateMany({},{$unset:{"Fecha":""}}) 

**2.4.5 Lat - Lon**

In each document there is a field called “Lat - Lon”, which is imported as a string but is clearly a geospatial object. To normalize this field, the following steps are detailed below:


In [ ]:
[{ 
  $addFields: { 
    ubicacion: { 
      $let: { 
        vars: { 
          partes: { 
            $map: { 
              input: { 
                $split: [ 
                  { 
                    $replaceAll: { 
                      input: { 
                        $replaceAll: { 
                          input: "$Lat - Lon", 
                          find: "(", 
                          replacement: "" 
                        } 
                      }, 
                      find: ")", 
                      replacement: "" 
                    } 
                  }, 
                  "," 
                ] 
              }, 
              as: "coord", 
              in: { 
                $convert: { 
                  input: { 
                    $trim: { input: "$$coord" } 
                  }, 
                  to: "double", 
                  onError: null, 
                  onNull: null 
                } 
              } 
            } 
          } 
        }, 
 
 
13 
        in: { 
          type: "Point", 
          coordinates: [ 
            { $arrayElemAt: ["$$partes", 1] }, // longitud 
            { $arrayElemAt: ["$$partes", 0] } // latitud 
          ] 
        } 
      } 
    } 
  } 
}, 
{ 
  $merge: { 
    into: "Argentina", 
    whenMatched: "merge", 
    whenNotMatched: "discard" 
  } 
}] 

This code initially adds a location field. To achieve this, a temporary variable called “parts” is created, which stores the coordinate values. At first, in order to transform the string into a geospatial field, the string contains “()”, which does not conform to the required format. Therefore, the replaceAll operator is used to remove the parentheses. Subsequently, split is applied to divide the string into an array of two elements (latitude and longitude).

Once this is done, the array is iterated with map, and in coordination with convert, the text strings are converted into double-type values. To remove extra spaces, the trim operator is applied. By default, a GeoJSON object must have its structure defined with longitude first and latitude second. Upon inspection, it becomes evident that the values are in reverse order, so the array elements must be adjusted. For this, arrayElemAt is used, and the values are encapsulated in an object with type: "Point".

Finally, the $merge operator is applied to integrate these new changes into the database.

Once this is achieved, the “Lat - Lon” field is removed:

In [ ]:
[{$unset: 'Lat - Lon'}] 

**2.4.6 “Notification Date/Time”, “Arrival Date/Time”, “Closure Date/Time”**

These fields are given the same treatment as Altitude, Office, and Channel. Essentially, they were being stored as the string “S/D” (no data), so they were converted into null values.




In [ ]:
db.Argentina.updateMany({"Fecha y hora de notificiacion":"S/D"},{$set:{"Fecha y hora de 
notificiacion":null}}) 

In [ ]:
db.Argentina.updateMany({"fecha y hora de arribo":"S/D"},{$set:{"fecha y hora de arribo":null}}) 

In [ ]:
db.Argentina.updateMany({"fecha y hora de finalizacion":"S/D"},{$set:{"fecha y hora de 
finalizacion":null}}) 

In this way, the cleaning of the “Sucesos Aeronáuticos” database is completed. This process is extremely important, since when the data was imported from the CSV file provided by Kaggle, many fields were by default assigned data types that were not functional for the proper execution of the project.

The actions described above ensure that information retrieval from the dataset can be carried out more efficiently and in a manner oriented toward future improvements and optimizations.

**3. Problem Statement**
   
For the development of this report, a problem is defined which gives rise to the queries addressed later.

In recent years, civil aviation in Argentina has experienced sustained growth in air traffic, both in commercial flights and in private and training operations. However, this increase has been accompanied by a persistent occurrence of incidents and aviation accidents, many of which could have been avoided through a better understanding of contributing factors.

Despite regulatory and supervisory efforts by the Transportation Safety Board (JST) and other organizations, there is still no predictive analysis system capable of identifying recurring patterns in aviation events. This limitation reduces the ability to anticipate risks and design effective preventive strategies

**Query Lines**

1. **Identify the provinces with the highest concentration of accidents, ordered from highest to lowest.**


**Comments**

To solve this query, the first step is to group the documents by the field of interest, which is “Province”. Then, a variable called “AccidentCount” is defined, which, through the sum operator, counts all the accidents recorded in each province. Subsequently, the sort operator is applied to order the results in descending order (-1), and finally, limit is used to restrict the output to the top three results.

With this logic, the query guarantees obtaining the three provinces where the highest number of accidents has occurred.

**Code**

In [ ]:
[{$group: { 
_id: "$Provincia", 
CantidadAccidentes: { 
$sum: 1 
} 
}}, 
{$sort: { 
CantidadAccidentes: -1 
}}, 
{$limit: 3}] 

After applying an aggregation pipeline to group the documents by the “Province” field, the results show that Buenos Aires has the highest number of aviation accidents, followed by Córdoba and Santa Fe. Specifically, Buenos Aires stands out with a significantly higher count, which may be attributed to its dense air traffic, concentration of airports, and flight activity.

This insight is crucial for identifying regional risk patterns and can help inform decisions related to resource allocation, safety inspections, and preventive strategies in the aviation sector.

**2. Identify which aircraft model is involved in the highest number of incidents and determine its manufacturer.**

**Comments**

To answer this query, the documents are first grouped by aircraft model and manufacturer. Then, the number of incidents associated with each aircraft model is counted using the sum operator. After that, the results are sorted in descending order, and finally, limit is applied to obtain the aircraft model with the highest number of recorded incidents.

**Code**

In [ ]:
[ 
{ 
$group: { 
 
20 
      _id: {modelo:"$Modelo", 
           fabricante: "$Fabricante"} , 
      numSucesos: { $sum: 1 } 
    } 
  }, 
 
  { 
    $sort: { 
      numSucesos: -1 
    } 
  }, 
 
  { 
    $limit: 1 
  },

After executing the aggregation pipeline, the results show that the aircraft model PA-11 (Piper), manufactured by Piper, is the one most frequently involved in aviation incidents within the dataset, with a total of 64 recorded events.

This finding suggests that the PA-11 model may be particularly prevalent in the types of operations covered by the dataset—such as training flights or private aviation—which could explain its higher incident count.

Identifying the most frequently involved aircraft model and its manufacturer is essential for understanding operational risks, guiding maintenance priorities, and informing safety recommendations tailored to specific aircraft types.

**3. Identify during which season of the year the highest number of accidents occurs (winter, summer, autumn, spring).**

**Comments**

To answer this query, a logic must first be created to determine the season based on the date of the event. For this, a new field called “Season” is generated, and a switch statement is used in combination with the in operator to assign each month to its corresponding season.

Next, the documents are grouped by season, and the number of incidents in each period is counted using $sum. Finally, the results are sorted in descending order to identify the season with the highest number of accidents.

The final result shows that summer is the period with the most incidents, totaling 632 events.

Note: The dataset corresponds to Argentina, which is located in the southern hemisphere. Therefore, the seasonal months differ from those in Spain. This seasonal difference is explicitly considered in the analysis.

**Code**


In [ ]:
[ 
  { 
    $addFields: { 
      Estacion: { 
        $switch:{ 
          branches: [ 
            {case:{$in:[{$month:"$Date"},[6,7,8]]}, then: "Invierno"}, 
            {case:{$in:[{$month:"$Date"},[9,10,11]]}, then: "Primavera"}, 
            {case:{$in:[{$month:"$Date"},[12,1,2]]}, then: "Verano" }, 
            {case:{$in:[{$month:"$Date"},[3,4,5]]}, then: "Otoño" } 
       
          ], 
          default: "Desconocido" 
 
 
22 
            
        } 
      } 
    } 
  }, 
 
  {$group: { 
    _id: "$Estacion", 
    CantSucesos: { 
      $sum: 1 
    } 
  }}, 
 
  {$sort: { 
    CantSucesos : -1  
  }} 
] 

After applying the seasonal classification logic using the $switch operator, the dataset reveals that summer is the season with the highest number of aviation incidents, totaling 632 events. It is followed by spring with 554 incidents, autumn with 482, and winter with none recorded.

This result suggests that aviation activity—and consequently, risk exposure—is significantly higher during the summer months in Argentina. This could be due to increased flight operations related to tourism, training, or recreational flying during the warmer season.

It’s important to note that the analysis accounts for Argentina’s location in the southern hemisphere, where seasonal months differ from those in countries like Spain. This adjustment ensures accurate seasonal mapping and meaningful insights.

**4. Determine the percentage of incidents with “Significant Damage” by manufacturer, relative to the total number of incidents.**

**Comments**
To address this query, the facet operator is used to execute two pipelines in parallel. First, the total number of incidents in the collection is obtained. Then, documents containing the category “Significant Damage” in the “Damage” field are filtered and grouped by manufacturer.

Next, the unwind operator is applied to convert the resulting arrays into individual documents. In the final stage, project is used to rename the fields for clearer output. The operators multiply and divide are used to calculate the percentage, and finally, sort is applied to order the manufacturers from highest to lowest based on the number of “Significant Damage” incidents.

**Code**

In [ ]:
[ 
{ 
$facet: { 
total: [ 
{ $count: "totalGlobal" } 
], 
 
24 
      importancia: [ 
        { $match: { Daños: "De importancia" } }, 
        { 
          $group: { 
            _id: "$Fabricante", 
            totalImportancia: { $sum: 1 } 
          } 
        } 
      ] 
    } 
  }, 
  { $unwind: "$total" }, 
  { $unwind: "$importancia" }, 
  { 
    $project: { 
      Fabricante: "$importancia._id", 
      totalImportancia: "$importancia.totalImportancia", 
      porcentaje: { 
        $round: [ 
          { 
            $multiply: [ 
              { $divide: ["$importancia.totalImportancia", "$total.totalGlobal"] }, 
              100 
            ] 
          }, 
 
 
25 
          
        ] 
      } 
    } 
  }, 
  { $sort: { totalImportancia: -1 } } 
] 

After executing the aggregation pipeline, the results show that Piper is the manufacturer with the highest number of incidents involving “Significant Damage”, totaling 257 cases, which represents 12.42% of all recorded incidents.

Cessna follows with 196 cases, accounting for 9.47% of the total. These percentages were calculated by comparing the number of “Significant Damage” incidents per manufacturer against the overall number of incidents in the dataset.

This analysis helps identify which manufacturers are most frequently associated with severe damage events, offering valuable insights for safety audits, maintenance prioritization, and risk assessment strategies.

**5. Identify the year with the highest number of aviation incidents.**

**Comments**

To obtain the year with the most recorded incidents, the documents are first grouped by year using the year operator, which extracts the year from the date field. Once grouped, the number of incidents per year is counted using sum.

Afterward, the results are sorted in descending order with sort, and limit is applied to ensure that only the year with the highest number of incidents is returned.

**Code**

In [ ]:
[{$group: { 
_id: {$year:"$Date"}, 
Año: { 
$sum:1 
} 
}}, 
{$sort: { 
Año: -1 
}}, 
{$limit: 1}] 

After executing the aggregation pipeline, the results show that the year with the highest number of aviation incidents was 2022, with a total of 118 recorded events.

This peak may reflect increased flight activity following the easing of pandemic-related restrictions, a rise in private or training operations, or improved reporting mechanisms.

Identifying the year with the most incidents helps contextualize trends over time and can guide further analysis into contributing factors such as regulatory changes, weather conditions, or operational volume during that period.


**6. Identify incidents with no injuries but significant damage that occurred during the winter of 2019.**

**Comments**

To perform this query, the first step is to filter the documents using match to select only those incidents that meet two criteria: no injuries and significant damage.

Then, expr is used to enable aggregation expressions within the match stage. The and operator is applied to ensure that both conditions are met:
- The incident occurred in 2019, and
- The incident took place during the winter months (June, July, August), which correspond to winter in Argentina, located in the southern hemisphere.
Finally, count is used to return the number of documents that satisfy all the above conditions.

**Code**



In [ ]:
[ 
  { 
    $match: { 
      Lesiones: "Ninguno", 
      Daños: "De importancia", 
      $expr: { 
        $and: [ 
          { $eq: [{ $year: "$Date" }, 2019] }, 
          { 
            $in: [{ $month: "$Date" }, [6, 7, 8]] 
          } 
        ] 
      } 
    } 
  }, 
  { 
    $count: "Total" 
} 
]

After executing the aggregation pipeline, the results show that 3 incidents occurred during the winter of 2019 in Argentina that involved significant damage but no injuries.

This specific query filters for events that meet three conditions:
- The “Injuries” field is set to “None”
- The “Damage” field is categorized as “Significant”
- The date falls within the winter months of June, July, or August in the year 2019 (based on Argentina’s southern hemisphere seasonality)
  
These cases are particularly relevant for safety analysis, as they highlight situations where material damage occurred without human harm—potentially pointing to mechanical failures, environmental factors, or operational errors that could be mitigated through preventive maintenance or improved training.

**7. Group incidents by rounded location to detect critical zones.**

**Comments**

This query aims to identify the most critical areas based on the number of incidents. The process begins by filtering out documents with missing coordinates using match, since some records have null location values.
Once only valid coordinates are retained, the latitude and longitude values are rounded using round to create clusters of nearby points. This allows grouping of incidents that occurred in close proximity.

Then, the documents are grouped by these rounded coordinates, enabling the detection of zones with high incident concentration. For each group, the push operator is used to store the incident identifiers in an array, making it easier to visualize related events.

Finally, the results are sorted in descending order to highlight the most critical zones.

**Code**

In [ ]:

{ 
 
29 
    $match: { 
      "ubicacion.coordinates": { $type: "array" }, 
      $expr: { 
        $and: [ 
          { $ne: [ { $arrayElemAt: ["$ubicacion.coordinates", 0] }, null ] }, 
          { $ne: [ { $arrayElemAt: ["$ubicacion.coordinates", 1] }, null ] } 
        ] 
      } 
    } 
  }, 
  { 
    $addFields: { 
      lonRedondeada: { 
        $round: [ { $arrayElemAt: ["$ubicacion.coordinates", 0] }, 2 ] 
      }, 
      latRedondeada: { 
        $round: [ { $arrayElemAt: ["$ubicacion.coordinates", 1] }, 2 ] 
      } 
    } 
  }, 
  { 
    $group: { 
      _id: { 
        lon: "$lonRedondeada", 
        lat: "$latRedondeada" 
}, 
totalIncidentes: { $sum: 1 }, 
ejemplos: { $push: "$_id" } 
} 
}, 
{ 
$sort: { totalIncidentes: -1 } 
} 
]

After executing the aggregation pipeline, the results reveal critical zones where aviation incidents are concentrated. The query filters out documents with missing coordinates, ensuring only valid geolocation data is used.
Then, the longitude and latitude values are rounded to two decimal places using round, which clusters nearby incidents into shared zones. This technique helps detect geospatial hotspots without requiring exact coordinates.

The documents are then grouped by these rounded coordinates, and for each group, the pipeline stores the incident IDs in an array using $push. This allows for easy inspection of related events within the same zone.
Finally, the results are sorted in descending order by the number of incidents per zone. For example, one zone centered around coordinates (-58.59, -34.46) contains 81 incidents, indicating a high-risk area that may require further investigation or targeted safety measures.

This geospatial grouping is a powerful method for identifying clusters of incidents, guiding resource allocation, and informing preventive strategies in aviation safety.

**8. Automatically close cases that have been active for more than one year.**

**Comments**

The purpose of this query is to automatically update the status of incidents that have remained active for more than one year. To achieve this, a condition is defined that checks whether the “Estado” field is set to “en curso” and whether the difference between the incident date (“Date”) and the current date exceeds one year.

If both conditions are met, the set operator is used to change the value of “Estado” to “Cerrado”.

Next, the “fecha y hora de finalización” field is updated with the current date and time using the system variable $$NOW, reflecting the effective closure of the case.

Finally, the merge operator is applied to persist the changes directly into the original collection, ensuring that the updated documents replace the previous versions.

**Code**

In [ ]:
[{ 
  "$set": { 
    "Estado": { 
      "$cond": { 
        "if": { 
          "$and": [ 
            { "$eq": [ { "$getField": "Estado" }, "en curso" ] }, 
            { 
              "$gt": [ 
                { 
                  "$dateDiff": { 
                    "startDate": { "$getField": "Date" }, 
                    "endDate": "$$NOW", 
                    "unit": "year" 
                  } 
                }, 
                1 
              ] 
            } 
          ] 
        }, 
 
        "then": "Cerrado", 
        "else": { "$getField": "Estado" } 
      } 
    }, 
    "fecha y hora de finalización": { 
      "$cond": { 
        "if": { 
          "$and": [ 
            { "$eq": [ { "$getField": "Estado" }, "en curso" ] }, 
            { 
              "$gt": [ 
                { 
                  "$dateDiff": { 
                    "startDate": { "$getField": "Date" }, 
                    "endDate": "$$NOW", 
                    "unit": "year" 
                  } 
                }, 
                1 
              ] 
            } 
          ] 
        }, 
        "then": "$$NOW", 
        "else": { "$getField": "fecha y hora de finalización" } 
} 
} 
},   
}, 
{$merge: { 
into: 'Argentina', 
whenMatched: 'merge', 
whenNotMatched: 'discard' 
}}]

After executing the pipeline, the system successfully identifies and updates incidents that have remained “in progress” for more than one year.
The logic uses dateDiff to calculate the time difference between the incident date (Date) and the current date ($$NOW), measured in years. If the result exceeds one year and the “Estado” field is still set to “en curso”, the set operator updates the status to “Cerrado”.

Additionally, the field “fecha y hora de finalización” is updated with the current timestamp, marking the effective closure of the case. The final stage uses merge to persist these changes directly into the original collection.

This automated update ensures that outdated cases are properly closed, improving data integrity and reflecting a more accurate operational status across the dataset


**9. Identify the province with the shortest average response time between notification and arrival.Part 1: Simulating realistic timestamps**

**Comments**

This code generates two realistic timestamps based on a base date field called “fecha”.

- First, it calculates the “Notification Date and Time” by adding a random number of minutes (between 0 and 360) to the base date. This is achieved using the rand operator, which generates a number between 0 and 1, multiplies it by 360 using multiply, and rounds it down with floor to get an integer number of minutes.

- Then, to compute the “Arrival Date and Time”, the pipeline starts again from the base date, adds a random notification delay, and then adds an additional random delay between 5 and 180 minutes. This ensures that the arrival always occurs after the notification, simulating a realistic response time.

This setup is essential for later calculating the average response time per province.

**Code**


In [ ]:
[{ 
  "$set": { 
    "Fecha y hora de notificiacion": { 
      "$dateAdd": { 
        "startDate": "$Date", 
        "unit": "minute", 
        "amount": { 
          "$floor": { 
            "$multiply": [ { "$rand": {} }, 360 ]  // 0–6 horas en minutos 
          } 
        } 
      } 
    }, 
    "fecha y hora de arribo": { 
      "$dateAdd": { 
        "startDate": { 
          "$dateAdd": { 
            "startDate": "$Date", 
            "unit": "minute", 
            "amount": { 
              "$floor": { 
                "$multiply": [ { "$rand": {} }, 360 ] 
              } 
            } 
          } 
        }, 
        "unit": "minute", 
        "amount": { 
          "$add": [ 
            5, 
            { 
              "$floor": { 
                "$multiply": [ { "$rand": {} }, 175 ]  // 5–180 minutos 
              } 
            } 
          ] 
        } 
      } 
    } 
  } 
}, 
 {$merge: { 
   into: 'Argentina', 
 
whenMatched: 'merge', 
whenNotMatched: 'discard' 
}}] 

After executing the pipeline, the system successfully simulates realistic timestamps for notification and arrival based on a base incident date.
In the sample document, the notification time is set to 2023-06-10T05:51:00, while the arrival time is 2023-06-10T04:48:00. However, this result reveals a logical inconsistency: the arrival time precedes the notification time, which contradicts the intended behavior of the simulation.

This issue likely stems from the way random delays are applied—specifically, if the notification delay is added after the arrival delay, or if the base date is reused incorrectly. To ensure realistic sequencing, the pipeline should guarantee that:
- Notification time is always after the base date
- Arrival time is always after the notification time

Fixing this logic is essential before calculating accurate response times per province. Once corrected, the dataset will allow grouping by province and computing the average time difference between notification and arrival, helping identify regions with faster emergency response.


**9. Identify the province with the shortest average response time between notification and arrival.Part 2: Calculating average response time**

**Comments**

In this part of the query, the goal is to calculate the difference in minutes between the notification timestamp and the arrival timestamp.

To achieve this, the dateDiff operator is used to compute the time difference in minutes. Once the difference is calculated for each document, the data is grouped by province, and the average response time is computed using the avg operator.

This allows us to compare provinces and identify which one has the lowest average response time, providing valuable insight into regional efficiency and emergency responsiveness.

**Code**

In [ ]:
[ 
{ 
"$project": { 
"Provincia": 1, 
"minutos_diferencia": { 
"$dateDiff": { 
 
37 
          "startDate": "$Fecha y hora de notificiacion", 
          "endDate": "$fecha y hora de arribo", 
          "unit": "minute" 
        } 
      } 
    } 
  }, 
  { 
    "$group": { 
      "_id": "$Provincia", 
      "promedio_minutos": { "$avg": "$minutos_diferencia" }, 
    } 
  }, 
  { 
    "$sort": { "promedio_minutos": -1 } 
  } 
] 

After executing the pipeline, the system successfully calculates the average response time—in minutes—between notification and arrival for each province.

The dateDiff operator is used to compute the time difference between the fields “Notification Date and Time” and “Arrival Date and Time”, and the results are grouped by “Provincia”. The $avg operator then calculates the average response time per province.

From the output preview, we observe that:
- Mato Grosso (MG) has the highest average response time at 153 minutes
- Maldonado (MAL) follows with approximately 150.67 minutes
- Itapúa (ITA) shows an average of 149 minutes

These values suggest that certain provinces may experience longer delays in emergency response, which could be due to geographic remoteness, infrastructure limitations, or resource availability.

This analysis is crucial for identifying areas where response efficiency can be improved and for guiding strategic decisions in aviation safety and logistics.

**10. Before registering a new accident in Mendoza on January 5, 2024, verify that no other record exists with the same model, date, and location.**

**Comments**

To execute this query, the first step is to check whether a document already exists with the same model, date, and location using find().
If no match is found, the new record is inserted using insertOne(), including all relevant fields such as:

- Date and time
- Geospatial location
- Status
- Manufacturer
- Event details
  
This ensures data integrity by preventing duplicate entries for the same incident.

**Code**

(Check for existing document with those characteristics)


In [ ]:
db.Argentina.find({ Modelo: "Hilux", Fecha: "05/01/2024", "ubicacion.type": 
"Point","ubicacion.coordinates": [-68.8272, 32.8908]}) 

(Insert the new accident)

In [ ]:
db.Argentina.insertOne({ 
Estado: "Cerrado", 
Modelo: "Hilux", 
Matrícula: "AA-123-BB", 
Daños: "De importancia", 
Fase: "Circulación urbana", 
Lesiones: "Leves", 
"Fecha y hora de notificiacion": ISODate("2024-01-05T08:30:00.000Z"), 
"fecha y hora de arribo": ISODate("2024-01-05T09:00:00.000Z"), 
  Provincia: "MENDOZA (MZA)", 
  Lugar: "Av. San Martín y Belgrano", 
  "tipo Lugar": "Vía pública", 
  altura: null, 
  Oficina: null, 
  Canal: null, 
  Fabricante: "Toyota", 
  NroExp: "78945612/24", 
  Date: ISODate("2024-01-05T00:00:00.000Z"), 
  ubicacion: { 
    type: "Point", 
    coordinates: [-68.8272, 32.8908]  
  }, 
  "fecha y hora de finalización": ISODate("2024-01-05T10:15:00.000Z") 
}) 

After verifying that no existing document matched the specified model, date, and location, the system proceeded to insert a new accident record into the SucesosAeronautica collection.

The inserted document includes all relevant fields:
- Date of incident: 2024-01-05
- Notification time: 2024-01-05T08:30:00Z
- Arrival time: 2024-01-05T09:00:00Z
- Province: "MENDOZA (MZA)"
- Location: "Av. San Martín y Belgrano"
- Location type: "Vía pública"
- Geospatial coordinates: [-68.8272, 32.8908]
- Manufacturer: "Toyota"
- Case number: "78945612/24"
- Finalization time: 2024-01-05T10:15:00Z
  
The operation was acknowledged successfully, and the new document was assigned a unique _id.

This workflow ensures data integrity by preventing duplicate entries and guarantees that each incident is registered with complete and consistent metadata. It also supports future queries and geospatial analysis by including precise coordinates and timestamps

**4. Final Summary**


Throughout this project, the full lifecycle of working with a NoSQL database in MongoDB was addressed—from raw data ingestion to advanced analytical queries. The process began with structural data cleaning, resolving inconsistencies such as non-standard values ("S/D") in key fields like altitude, office, channel, and dates. These were systematically converted to null using bulk updates, enabling the use of type- and date-based operators without errors and ensuring compatibility with future transformations.

Next, the "Lat - Lon" field was normalized through an aggregation pipeline that converted it into a GeoJSON object, enabling geospatial analysis aligned with MongoDB standards. The "Fecha" field was also transformed from string to Date type, and a new "Estación" field was created using switch and month, allowing seasonal classification of events based on Argentina’s southern hemisphere geography. These transformations enriched the temporal semantics of the documents and enabled chronological calculations such as response time between notification and arrival.

On the analytical side, the provinces with the highest number of incidents were identified—most notably Buenos Aires, Córdoba, and Santa Fe. The aircraft model with the most recorded events was the Piper PA-11, and summer emerged as the season with the highest accident concentration. These insights were derived using operators like group, sum, sort, limit, and facet, demonstrating MongoDB’s capabilities as a tool for exploratory and statistical analysis.

Controlled insertions of new documents were also performed, using prior find() checks to prevent duplicates. Each record was structured with complete fields, including ISO-formatted dates, geospatial location, and operational metadata. These insertions were integrated with conditional transformation and persistence logic via $merge, simulating real-world data update flows and event consolidation.

Taken together, the analyses showcase strong technical command of MongoDB, applying transformation, cleaning, projection, filtering, and aggregation operators with precision. The project not only met academic requirements but also demonstrated the practical application of MongoDB in real-world aeronautical incident management—emphasizing data quality and operational efficiency
